I chose route based on how close I am to target mileage and how much I like the two paths.
If I am almost done... I'll take a less liked path that's toward home than one that is more liked that's away from home.
I try not to turn around and go back the way I came (if possible).
State consists of current location, possible next moves, and how complete the run is.
Descision depends on the state paramters AND the reward associated with next and future moves. 


In [79]:
# import libraries
import numpy as np
import random

In [47]:
# these are the possible moves
splits = [(0,1), (0,2), (1,3), (1,4), (2,5), (2,6), (3,4), (3,7),\
          (4,3), (4,5), (4,7), (5,4), (5,6), (5,8), (6,5), (6,8),\
          (7,3), (7,4), (7,9), (8,5), (8,6), (8,9), (9,7), (9,8)]

# adapt the reward structure to consider state of completion
# completion_rate_list = [i/10.0 for i in range(0, 13, 2)]
completion_rate_list = [0.5,1.0]

In [48]:
completion_rate_list

[0.5, 1.0]

In [49]:
# create rewards look-up table
rtable = {}
for i in splits:
    for j in completion_rate_list:
        key = "{}, {}".format(i, j)
        rtable[key] = 1

In [55]:
rtable

{'(0, 1), 0.5': 0.0,
 '(0, 1), 1.0': 0.0,
 '(0, 2), 0.5': 0.5,
 '(0, 2), 1.0': 0.5,
 '(1, 3), 0.5': 1,
 '(1, 3), 1.0': 1,
 '(1, 4), 0.5': 1,
 '(1, 4), 1.0': 1,
 '(2, 5), 0.5': 1,
 '(2, 5), 1.0': 1,
 '(2, 6), 0.5': 1,
 '(2, 6), 1.0': 1,
 '(3, 4), 0.5': 1,
 '(3, 4), 1.0': 1,
 '(3, 7), 0.5': 1,
 '(3, 7), 1.0': 1,
 '(4, 3), 0.5': 1,
 '(4, 3), 1.0': 1,
 '(4, 5), 0.5': 2.0,
 '(4, 5), 1.0': 2.0,
 '(4, 7), 0.5': 1,
 '(4, 7), 1.0': 1,
 '(5, 4), 0.5': 2.0,
 '(5, 4), 1.0': 2.0,
 '(5, 6), 0.5': 1,
 '(5, 6), 1.0': 1,
 '(5, 8), 0.5': 1,
 '(5, 8), 1.0': 1,
 '(6, 5), 0.5': 1,
 '(6, 5), 1.0': 1,
 '(6, 8), 0.5': 1,
 '(6, 8), 1.0': 1,
 '(7, 3), 0.5': 1,
 '(7, 3), 1.0': 1,
 '(7, 4), 0.5': 1,
 '(7, 4), 1.0': 1,
 '(7, 9), 0.5': 1,
 '(7, 9), 1.0': 100.0,
 '(8, 5), 0.5': 1,
 '(8, 5), 1.0': 1,
 '(8, 6), 0.5': 1,
 '(8, 6), 1.0': 1,
 '(8, 9), 0.5': 1,
 '(8, 9), 1.0': 100.0,
 '(9, 7), 0.5': 10.0,
 '(9, 7), 1.0': 1,
 '(9, 8), 0.5': 10.0,
 '(9, 8), 1.0': 1}

In [95]:
# alter rewards as necessary in table
for i, j in rtable.iteritems():
    if '0, 1' in i:
        rtable[i] = 0.0
    elif '0, 2' in i:
        rtable[i] = 0.5
    elif '4, 5' in i or '5, 4' in i:
        rtable[i] = 50.0

for i, j in rtable.iteritems():
    if i=='(7, 9), 1.0' or i=='(8, 9), 1.0': rtable[i] = 500.0
            
for i, j in rtable.iteritems():
    if i=='(9, 7), 0.5' or i=='(9, 8), 0.5': rtable[i] = 10.0
        
# for i, j in rtable.iteritems():
#     if i=='(7, 9), 1.0' or i=='(8, 9), 1.0' or i=='(7, 9), 1.2' or i=='(8, 9), 1.2':
#         rtable[i] = 1000

In [17]:
# rtable

In [56]:
mtable = {}
for i in splits:
    key = "{}".format(i)
    mtable[key] = 0

In [57]:
for i, j in mtable.iteritems():
    if '0, 1' in i or '3, 4' in i or '4, 3' in i or '5, 6' in i or '6, 5' in i or '7, 9' in i or '9, 7' in i:
        mtable[i] = 0.50
    elif '1, 3' in i or '3, 7' in i or '7, 3' in i:
        mtable[i] = 0.25
    elif '4, 5' in i or '5, 4' in i:
        mtable[i] = 0.75
    elif '0, 2' in i or '8, 9' in i or '9, 8' in i:
        mtable[i] = 1.0
    elif '1, 4' in i or '2, 5' in i or '4, 7' in i or '5, 8' in i or '7, 4' in i or '8, 6' in i:
        mtable[i] = 0.35
    elif '2, 6' in i or '6, 8' in i or '8, 5' in i:
        mtable[i] = 0.10

In [96]:
# create q look-up table
q = {}
for i in splits:
    for j in completion_rate_list:
        key = "{}, {}".format(i, j)
        q[key] = 0

In [18]:
for i in range(1):
    # starting metrics
    # gamma, alpha, epsilon
    gamma = 0.8; alpha = 1; epsilon = 0.6
    # current state = 0
    current_state = 0
    # target miles and total miles (starts at 0)
    target_miles = 5; current_total_miles = 0
    # set random state for replication
    random_state = np.random.RandomState(1999)
    # set goal to False
    goal = False
    # loop until run is over
    while not goal:
        # define next possible states (valid_moves)
        valid_moves = [key for key, value in mtable.items() if key.lower()[1]==str(current_state)]
        # determine to exploit of not (epsilon > random number). Exploit or explore.
        if random.random() < epsilon:
        # if not exploiting...
            # define action (action)
            action = random.choice(valid_moves)
        # if exploiting
        else:
            # define action (action & transition)
            exploit_vals = [value for key, value in q.items() if key.lower()[1]==str(current_state)]
            exploit_keys = [key for key, value in q.items() if key.lower()[1]==str(current_state)]
            if sum(exploit_vals) > 1:
                action = exploit_keys[exploit_vals.index(max(exploit_vals))][:6]       
            else:
                action = random.choice(valid_moves)
        # define transition
        transition = action #+ ", " + str(pct_complete)
        # determine reward (reward)
        reward = rtable[action]
        # determine standing q value
        qval = q[action]
        # update q value (new_qvals)
        next_qvals = [value for key, value in q.items() if key.lower()[1]==str(action[1])]
        new_qval = qval + alpha * (reward + gamma * max(next_qvals) - qval)
        # set q val as new q val
        q[transition] = new_qval
        # increment mileage (addmiles)
        current_total_miles += mtable[action]
        # set current state to previous action
        current_state = action[4]
        # if run is longer than we want, go to the end
        if current_total_miles >= target_miles and current_state==9:
            goal = True
    print "RUN COMPLETE!!!"
    print "TOTAL MILES: {}".format(current_total_miles)

KeyError: '(0, 2)'

In [10]:
# starting metrics
# gamma, alpha, epsilon
# current state = 0
# target miles and total miles (starts at 0)
# set random state for replication


# define current state (current_state)
# calculate percent of run complete (pct_complete)
# define next possible states (valid_moves)
# determine to exploit of not (epsilon > x)
# define action (action & transition)
# determine reward (reward)
# update q value (new_qvals)
# increment mileage (addmiles)
# set current state to previous action

In [80]:
#def roundPartial (value, resolution):
#    return round (value / resolution) * resolution
def roundPartial(number):
    return 0.5 * np.ceil(2.0 * number)

In [81]:
roundPartial(.1)

0.5

In [900]:
# create q look-up table
q = {}
for i in splits:
    for j in completion_rate_list:
        key = "{}, {}".format(i, j)
        q[key] = 0

In [1561]:
    gamma = 0.8; alpha = 1; epsilon = 0.8
    # current state = 0
    current_state = 0
    # target miles and total miles (starts at 0)
    target_miles = 5; current_total_miles = 0
    # set random state for replication
    random_state = np.random.RandomState(1999)

In [1650]:
pct_complete = roundPartial((current_total_miles/target_miles), 0.2)
print pct_complete

1.0


In [1651]:
valid_moves = [key for key, value in mtable.items() if key.lower()[1]==str(current_state)]
print valid_moves

['(4, 5)', '(4, 7)', '(4, 3)']


In [1652]:
if rand_num < epsilon:
    print random.random()
# if not exploiting...
    print "random"
    # define action (action)
    action = random.choice(valid_moves)
# if exploiting
else:
    # define action (action & transition)
    exploit_vals = [value for key, value in q.items() if key.lower()[1]==str(current_state)]
    exploit_keys = [key for key, value in q.items() if key.lower()[1]==str(current_state)]
    if sum(exploit_vals) > 1:
        action = exploit_keys[exploit_vals.index(max(exploit_vals))][:6]       
    else:
        print "not enough info to exploit"
        action = random.choice(valid_moves)
print action

0.520786730132
random
(4, 5)


In [1653]:
transition = action + ", " + str(pct_complete)

In [1654]:
reward = rtable[transition]
print reward

2.0


In [1655]:
qval = q[transition]
print qval

0


In [1656]:
next_qvals = [value for key, value in q.items() if key.lower()[1]==str(action[4])]
print next_qvals

[0, 0, 0, 0, 0, 0, 0, 6.7666366940446725, 0, 0, 0, 0, 4.9478698344448, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4.934837293056001, 0]


In [1657]:
new_qval = qval + alpha * (reward + gamma * max(next_qvals) - qval)
print new_qval

7.41330935524


In [1658]:
q[action] = new_qval
print q[action]

7.41330935524


In [1659]:
current_total_miles += mtable[action]
print current_total_miles

5.55


In [1660]:
current_state = action[4]
print current_state

5


In [31]:
# create q look-up table
q = {}
for i in splits:
    for j in completion_rate_list:
        key = "{}, {}".format(i, j)
        q[key] = 0

In [93]:
# set random state for replication
random_state = np.random.RandomState(1999)

for i in range(100):
    gamma = 0.8; alpha = 1; epsilon = 0.8
    # current state = 0
    current_state = 0
    # target miles and total miles (starts at 0)
    target_miles = 5; current_total_miles = 0
    # run while loop per run
    goal = False
    while not goal:
        pct_complete = roundPartial((current_total_miles+0.01)/target_miles)
        print "Percent of run complete: " + str(pct_complete) 
        if pct_complete > 1.2:
            break
        valid_moves = [key for key, value in mtable.items() if key.lower()[1]==str(current_state)]
        print "Valid moves are: " + str(valid_moves)
        if random.random() < epsilon:
            # if not exploiting... we are exploring
            print "Randomly exploring options"
            # define action (action)
            action = random.choice(valid_moves)
            print "Chose randomly and action is: " + str(action)
        # if exploiting the info we already know
        else:
            # define action (action & transition)
            exploit_vals = [value for key, value in q.items() if key.lower()[1]==str(current_state)]
            exploit_keys = [key for key, value in q.items() if key.lower()[1]==str(current_state)]
            if sum(exploit_vals) > 1:
                action = exploit_keys[exploit_vals.index(max(exploit_vals))][:6] 
                print "Exploiting and Action is: " + str(action)
            else:
                action = random.choice(valid_moves)
                print "Not enough info to exploit... chossing randomly and Action is: " + str(action)
        transition = action + ", " + str(pct_complete)
        reward = rtable[transition]
        print "The reward for this action at this mileage is: " + str(reward)
        qval = q[transition]
        print "Existing qval for this transition: " + str(qval)
        next_qvals = [value for key, value in q.items() if key.lower()[1]==str(action[4])]
        print "Next options q vals: " + str(next_qvals)
        new_qval = qval + alpha * (reward + gamma * max(next_qvals) - qval)
        print "Updated q val for this transition: " + str(new_qval)
        q[transition] = new_qval
        #print q[transition]
        current_total_miles += mtable[action]
        print "Updated total current miles: " + str(current_total_miles)
        current_state = action[4]
        print "The current state: " + str(current_state)
    print "RUN COMPLETE"
    print "TOTAL MILES: {}".format(current_total_miles)

Percent of run complete: 0.5
Valid moves are: ['(0, 1)', '(0, 2)']
Randomly exploring options
Chose randomly and action is: (0, 1)
The reward for this action at this mileage is: 0.0
Existing qval for this transition: 0
Next options q vals: [0, 0, 0, 0]
Updated q val for this transition: 0.0
Updated total current miles: 0.5
The current state: 1
Percent of run complete: 0.5
Valid moves are: ['(1, 3)', '(1, 4)']
Randomly exploring options
Chose randomly and action is: (1, 3)
The reward for this action at this mileage is: 1
Existing qval for this transition: 0
Next options q vals: [0, 0, 0, 0]
Updated q val for this transition: 1.0
Updated total current miles: 0.75
The current state: 3
Percent of run complete: 0.5
Valid moves are: ['(3, 4)', '(3, 7)']
Randomly exploring options
Chose randomly and action is: (3, 7)
The reward for this action at this mileage is: 1
Existing qval for this transition: 0
Next options q vals: [0, 0, 0, 0, 0, 0]
Updated q val for this transition: 1.0
Updated total

In [94]:
q

{'(0, 1), 0.5': 1435.0331939610871,
 '(0, 1), 1.0': 0,
 '(0, 2), 0.5': 1435.5331939610871,
 '(0, 2), 1.0': 0,
 '(1, 3), 0.5': 1793.7945551688697,
 '(1, 3), 1.0': 0,
 '(1, 4), 0.5': 1793.7675462774614,
 '(1, 4), 1.0': 0,
 '(2, 5), 0.5': 1793.7914924513589,
 '(2, 5), 1.0': 0,
 '(2, 6), 0.5': 1793.7914924513589,
 '(2, 6), 1.0': 0,
 '(3, 4), 0.5': 1793.7945551688697,
 '(3, 4), 1.0': 1793.7675462774614,
 '(3, 7), 0.5': 2240.993193961087,
 '(3, 7), 1.0': 2240.993193961087,
 '(4, 3), 0.5': 1793.7914924513589,
 '(4, 3), 1.0': 1793.7945551688697,
 '(4, 5), 0.5': 1842.7867069552485,
 '(4, 5), 1.0': 1842.7914924513589,
 '(4, 7), 0.5': 2240.993193961087,
 '(4, 7), 1.0': 2240.993193961087,
 '(5, 4), 0.5': 1842.7914924513589,
 '(5, 4), 1.0': 1842.7914924513589,
 '(5, 6), 0.5': 1793.7867069552485,
 '(5, 6), 1.0': 1793.7914924513589,
 '(5, 8), 0.5': 2240.9893655641986,
 '(5, 8), 1.0': 2240.9833836940606,
 '(6, 5), 0.5': 1793.7914924513589,
 '(6, 5), 1.0': 1793.7914924513589,
 '(6, 8), 0.5': 2240.98936